In [138]:
from bs4 import BeautifulSoup
import requests
import datetime #as dt
from datetime import date 
from datetime import datetime
import pandas as pd 
import re 
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import sys
import warnings
from datetime import timedelta
import time as timp 
import dateutil.parser

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
#Get df of leagues 

def get_pinn_leagues(URL):
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(
        executable_path=chrome_path, options=chrome_options)
    browser.get(URL)
    #browser.implicitly_wait(20)

    timp.sleep(20)

    htmlSource = browser.page_source
    #browser.implicitly_wait(20)

    soup= BeautifulSoup(htmlSource, 'lxml')
    browser.quit()
    league_name= []
    links= []
    for element in soup.select('a[class*="style_supportFavoritesList"]'):
        links.append('https://www.pinnacle.com'+element['href'])
        league_name.append(element.text)
    
    link_frame= pd.DataFrame({
        'league': league_name,
        'link': links})
    
    p= re.compile('.*[^0-9]')
        
    for i in np.arange(0,len(link_frame)):
        link_frame['league'][i]= p.search(link_frame['league'][i]).group()
        
    return(link_frame)


In [139]:
def get_pinn_odds(URL,which):
    #import time as timp
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    #browser.implicitly_wait(10)
    browser.get(URL)
    timp.sleep(5)
    soup= BeautifulSoup(browser.page_source, 'lxml')
    browser.quit()
    df= pd.DataFrame()
    error= []

    df= pd.DataFrame()
    
    for element in soup.find_all('div',{'class':['style_dateBar__3gX3j','style_dateBar__2PS4O']}):
        count= len(element.text)
        if 'Today' in element.text:
            date= pd.to_datetime(datetime.now(), format="%d/%m/%Y")
        if 'Tomorrow' in element.text:
            days_add= timedelta(days= 1)
            date= (pd.to_datetime(datetime.now(),format="%d/%m/%Y")+days_add)
        else:
            if which== 'yes':
                try:
                    temp_date= dateutil.parser.parse(element.text.split(',',1)[1][1:].replace(',',''))
                    date= pd.to_datetime(temp_date, format= '%d/%m/%Y')
                except:
                    date= element.text
            else:
                 pass
        for tag in element.next_siblings:
            if tag['class'] == ['style_dateBar__3gX3j'] or tag['class']==['style_dateBar__2PS4O']:
                break
            if tag['class']== ['style_row__2ww2Y'] or tag['class']== ['style_row__2ww2Y', 'style_lastRow__3Y2le']\
            or tag['class']== ['style_row__3_aBC'] or tag['class']== ['style_row__3_aBC', 'style_lastRow__3h8Pm']:
                try:
                    teams= tag.find_all('span',{'class':["style_participantName__2tjoj ellipsis", "style_participantName__CNiJz ellipsis"]})
                    play_time= tag.find('span',{'class':["style_time__2GvnE ellipsis","style_time__1_zpO ellipsis"]})
                    odds= tag.find_all('span',{'class':'price'})
                    fest= pd.DataFrame({'date':date,
                                    'scrape_time':pd.to_datetime(datetime.now(), format="%d/%m/%Y %H:%M"),
                                    'time':play_time.text,
                         'home_team': teams[0].text,
                         'away_team': teams[1].text,
                         'PinnacleHome': odds[0].text,
                         'PinnacleDraw': odds[1].text,
                         'PinnacleAway': odds[2].text}, index= pd.Series(0))
                    
                    df= df.append(fest)                        
                except:
                    continue
    return(df.reset_index(drop= True))

# Get Pinnacle Odds

In [140]:
# Very Important script

#links= get_pinn_leagues('https://www.pinnacle.com/en/soccer/leagues')
links= pd.read_csv('links.csv')
results= pd.DataFrame()
errors= []

for i,element in enumerate(links['link']):
    try:
        df= get_pinn_odds(element,'yes')
        df['league']= links['league'][i]
        results= results.append(df)       
    except:
        errors.append(element) 

In [141]:
#links= get_pinn_leagues('https://www.pinnacle.com/en/soccer/leagues')

#get_pinn_odds(links['link'][2],'yes')
#links= links.drop_duplicates()


In [142]:
#soup.select('div[class*="style_dateBar"]')
#soup.find_all('div',{'class':'style_dateBar2PS40'})
#soup.find('span',{'class':["style_time__2GvnE ellipsis","style_time__1_zpO ellipsis"]})
#soup.find('div',{'class':["style_dateBar__3gX3j", "style_dateBar__2PS4O"]})
#style_dateBar__3gX3j

In [143]:
results.loc[results['time']== '24:00','time']= '00:00'
results.loc[results['time']== '24:15','time']= '00:15'
results.loc[results['time']== '24:30','time']= '00:30'
results.loc[results['time']== '24:45','time']= '00:45'


def change_date(x):
    if x == 'Today':
        return(datetime.strftime(datetime.now(), format="%Y-%m-%d"))
    else:
        return(datetime.strftime(x, '%Y-%m-%d'))   


results.scrape_time = results.scrape_time.apply(lambda x: x.strftime(format= "%Y-%m-%d %H:%M"))
results.scrape_time= pd.to_datetime(results.scrape_time)
results.date= results.date.map(change_date)
results.date= pd.to_datetime(results.date.astype(str)+' '+results.time.astype(str))
results['difference_to_scrape']= results.date-results.scrape_time
results.difference_to_scrape= results.difference_to_scrape.dt.total_seconds()/3600
results= results.drop('time',axis=1)
results[['PinnacleHome','PinnacleDraw','PinnacleAway']]= results[['PinnacleHome','PinnacleDraw','PinnacleAway']].astype('float')

probs_with_margin= (1/results['PinnacleHome']+1/results['PinnacleDraw']+1/results['PinnacleAway'])

results['ProbsPinnacleHome']= 1/results['PinnacleHome']/probs_with_margin
results['ProbsPinnacleDraw']= 1/results['PinnacleDraw']/probs_with_margin
results['ProbsPinnacleAway']= 1/results['PinnacleAway']/probs_with_margin

# Get Oddschecker Data

In [144]:
def scrape_odds(link):
    try:
        URL= link
        hdr = {'User-Agent': 'Mozilla/5.0'}
        page= requests.get(URL, headers= hdr)
        maps= page.text 
        soup= BeautifulSoup(maps, 'lxml')

# Bookmaker Names
        names= []
        for element in soup.find_all('a',{"class": "bk-logo-main-90 bk-logo-click"}):
            names.append(element['title'])
    
        names.insert(0, 'Outcome')

# The odds and Team Names 
        odds= []
        teams= []

        soup_odds= soup.find('tbody')

        for row in soup_odds.find_all('tr'):
            cells= row.find_all('td')
            outcome= cells[0].find('span').find('span')['data-name']
            opus= [x['data-odig'] for x in cells if x.has_attr('data-odig')]
            opus.insert(0, outcome)
            odds.append(opus)
            teams.append(outcome)

        date= soup.find('div',{'class':'event'}).text
        fixture= soup.find('h1').text
        scrape_time= pd.to_datetime(datetime.now(), format="%d/%m/%Y %H:%M:%S")

        data= pd.DataFrame(odds, columns= names)

        p= re.compile('^.+?(?=\ v)')

        home_team= p.search(fixture).group(0)#[:-1]

        lop= list(data['Outcome'])

        if home_team== lop[0] and lop[1] != 'Draw':
            order= [1,2,0]
            lop= [lop[i] for i in order]

        if home_team== lop[0] and lop[1] == 'Draw':
            order= [2,1,0]
            lop= [lop[i] for i in order]
            
        if home_team== lop[1] and lop[2] == 'Draw':
            order= [0,2,1]
            lop= [lop[i] for i in order]
            
        if home_team== lop[1] and lop[2] != 'Draw':
            order= [2,0,1]
            lop= [lop[i] for i in order]
            
        if home_team== lop[2] and lop[1] == 'Draw':
            order= [0,1,2]
            lop= [lop[i] for i in order]
            
        if home_team== lop[2] and lop[1] != 'Draw':
            order= [1,0,2]
            lop= [lop[i] for i in order]
    
        data['Outcome']= pd.Categorical(data['Outcome'], lop)
        data= data.sort_values('Outcome').reset_index(drop= True)

        a_list= list(data.columns[1:])
        b_list= ['H','D','A']

        oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]

        oo.sort()

        for i in np.arange(0,len(oo), 3):
            home= oo[i+2]
            away= oo[i]
            oo[i]= home
            oo[i+2]= away
 
        oo.insert(0, 'scrape_time')
        oo.insert(0, 'date')
        oo.insert(0, 'away_team')
        oo.insert(0,'home_team')

        new= pd.DataFrame(columns= oo, index= np.arange(0,1))

        new['home_team']= data['Outcome'][2]
        new['away_team']= data['Outcome'][0]

        date= soup.find('div',{'class':'event'}).text.split(',',1)[1][1:].\
        lower().replace("rd", "").replace("nd", "").replace("st", "").replace("th", "")

        num= [i for i,x in enumerate(date) if x.isspace()][1]

        date= date[:num]+ ' '+ str(scrape_time.year) + date[num:]

        date= pd.to_datetime(datetime.strptime(date, '%d %B %Y %H:%M'),format= "%Y-%m-d %H:%M")
        hours_added = timedelta(hours = 1)
        new['date']= date+hours_added
        new['scrape_time']= scrape_time

        for name, stuff in data.drop('Outcome',axis=1).iteritems():
            for snam, snugg in new.iteritems():
                if name in snam and snam.endswith('A'):
                    snugg[0]= stuff[0]
                if name in snam and snam.endswith('D'):
                    snugg[0]= stuff[1]         
                if name in snam and snam.endswith('H'):
                    snugg[0]= stuff[2]
            
        new[new.columns.difference(['home_team', 'away_team','date','scrape_time'])]= new[new.columns.difference(['home_team', 'away_team','date','scrape_time'])].astype('float')
        return(new)
    
    except:
        new= pd.DataFrame()
        return(new)

In [145]:
def get_leagues(link):
    lan= open('länder.txt','r').readlines()
    land= [x[:-1] for x in lan]
    URL= link
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page= requests.get(URL, headers= hdr)
    maps= page.text 
    soup= BeautifulSoup(maps, 'lxml')
    links= []
    for element in soup.find_all('loc'):
        s= element.text
        if s.count('/') < 6 and any(thing in s for thing in land) and not s.endswith('winner'):
            links.append(s)
    return(links)

In [146]:
#oddschecker.query('home_team == "Bayern Munich"')[homes].idxmax(axis= 1)
#links= get_leagues('https://www.oddschecker.com/sport/football/sitemap.xml')
#oddschecker
#links[0:3]

In [147]:
errors= []
oddschecker= pd.DataFrame()
links= get_leagues('https://www.oddschecker.com/sport/football/sitemap.xml')
for thing in links:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page= requests.get(thing, headers= hdr)
    maps= page.text 
    soup= BeautifulSoup(maps, 'lxml')
    if len(soup.findAll('td',{'class':"bh-date beta-caption2"})) > 2:  
        splitter= str(soup.findAll('td',{'class':"bh-date beta-caption2"})[2])
        soup= BeautifulSoup(str(soup).split(splitter)[0])    
    league= thing[thing.rindex('/')+1:]
    for element in soup.find_all('a',{'class':'beta-callout full-height-link whole-row-link'}):
        try:
            new= scrape_odds('https://www.oddschecker.com'+element['href'])
            new['league']= league
            oddschecker= oddschecker.append(new)
        except:
            errors.append(element) 
            
oddschecker= oddschecker.reset_index(drop= True)

# Merge the Data

In [148]:
display(oddschecker.head(10))
display(results.head(10))

,home_team,away_team,date,scrape_time,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,VBetH,VBetD,VBetA,William HillH,William HillD,William HillA,league,ToteH,ToteD,ToteA
0,Man City,Chelsea,2021-05-29 21:00:00,2021-05-08 21:22:36.251684,1.95,3.35,4.10,2.05,3.35,3.90,...,2.11,3.30,3.80,2.00,3.30,4.00,champions-league,NaN,NaN,NaN
1,Villarreal,Man Utd,2021-05-26 21:00:00,2021-05-08 21:22:39.810530,4.75,3.35,1.71,4.80,3.60,1.80,...,5.00,3.50,1.80,5.00,3.70,1.73,europa-league,NaN,NaN,NaN
2,Wolves,Brighton,2021-05-09 13:00:00,2021-05-08 21:22:41.640946,3.50,3.30,2.20,3.60,3.20,0.00,...,3.70,3.31,2.16,3.50,3.20,2.20,premier-league,NaN,NaN,NaN
3,Aston Villa,Man Utd,2021-05-09 15:05:00,2021-05-08 21:22:42.801473,4.45,3.75,1.80,4.40,3.75,1.80,...,4.40,3.79,1.83,4.50,3.70,1.80,premier-league,NaN,NaN,NaN
4,West Ham,Everton,2021-05-09 17:30:00,2021-05-08 21:22:43.579695,2.25,3.40,3.20,2.25,3.40,3.20,...,2.34,3.45,3.26,2.25,3.40,3.20,premier-league,NaN,NaN,NaN
5,Arsenal,West Brom,2021-05-09 20:00:00,2021-05-08 21:22:44.650348,1.43,4.65,7.00,1.47,4.40,6.90,...,1.47,4.60,7.30,1.44,4.40,7.00,premier-league,NaN,NaN,NaN
6,Fulham,Burnley,2021-05-10 21:00:00,2021-05-08 21:22:45.501020,2.20,3.40,3.30,2.20,3.35,3.35,...,2.23,3.47,3.36,2.25,3.30,3.30,premier-league,0.0,0.0,0.0
7,Bournemouth,Brentford,2021-05-17 19:00:00,2021-05-08 21:22:46.951098,0.00,0.00,0.00,0.00,0.00,0.00,...,2.41,3.45,2.93,2.40,3.40,2.80,championship,NaN,NaN,NaN
8,Barnsley,Swansea,2021-05-17 21:15:00,2021-05-08 21:22:49.381755,0.00,0.00,0.00,0.00,0.00,0.00,...,2.02,3.45,3.82,2.05,3.40,3.60,championship,NaN,NaN,NaN
9,Blackpool,Bristol Rovers,2021-05-09 13:00:00,2021-05-08 21:22:51.211499,1.64,3.80,5.50,1.63,3.85,5.20,...,1.66,3.89,5.40,1.65,3.75,5.50,league-1,NaN,NaN,NaN


,date,scrape_time,home_team,away_team,PinnacleHome,PinnacleDraw,PinnacleAway,league,difference_to_scrape,ProbsPinnacleHome,ProbsPinnacleDraw,ProbsPinnacleAway
0,2021-05-08 21:15:00,2021-05-08 20:56:00,Liverpool,Southampton,1.322,6.07,9.520,England - Premier League,0.316667,0.737105,0.160536,0.102359
1,2021-05-09 20:00:00,2021-05-08 20:56:00,Arsenal,West Brom,1.480,4.67,7.340,England - Premier League,23.066667,0.658522,0.208697,0.132781
2,2021-05-09 15:05:00,2021-05-08 20:56:00,Aston Villa,Manchester United,4.540,3.86,1.833,England - Premier League,18.150000,0.214916,0.252777,0.532307
3,2021-05-09 17:30:00,2021-05-08 20:56:00,West Ham United,Everton,2.350,3.46,3.260,England - Premier League,20.566667,0.416658,0.282990,0.300352
4,2021-05-09 13:00:00,2021-05-08 20:56:00,Wolves,Brighton and Hove Albion,3.720,3.35,2.190,England - Premier League,16.066667,0.262531,0.291527,0.445943
5,2021-05-10 21:00:00,2021-05-08 20:56:00,Fulham,Burnley,2.250,3.51,3.410,England - Premier League,48.066667,0.434622,0.278604,0.286774
6,2021-05-11 19:00:00,2021-05-08 20:56:00,Manchester United,Leicester City,2.280,3.28,3.320,England - Premier League,70.066667,0.419838,0.291839,0.288323
7,2021-05-11 21:15:00,2021-05-08 20:56:00,Southampton,Crystal Palace,2.070,3.68,3.370,England - Premier League,72.316667,0.459402,0.258414,0.282185
8,2021-05-12 21:15:00,2021-05-08 20:56:00,Chelsea,Arsenal,1.666,3.96,4.930,England - Premier League,96.316667,0.568622,0.239223,0.192155
9,2021-05-13 21:15:00,2021-05-08 20:56:00,Manchester United,Liverpool,3.430,3.72,2.040,England - Premier League,120.316667,0.277514,0.255880,0.466605


In [149]:
#Make column names the same 
oddschecker= oddschecker.sort_values('date').reset_index(drop= True)
results= results.sort_values('date').reset_index(drop= True)

for i in range(0, len(oddschecker)):
    for j in range (0,len(results)):
        if (oddschecker['date'][i]== results['date'][j]):
            if (oddschecker['home_team'][i] in results['home_team'][j])\
            | (results['home_team'][j] in oddschecker['home_team'][i])\
            | (results['away_team'][j] in oddschecker['away_team'][i])\
            | (oddschecker['away_team'][i] in results['away_team'][j]):
                results['home_team'][j]= oddschecker['home_team'][i]
                results['away_team'][j]= oddschecker['away_team'][i]

In [150]:
#odds_merged.query('(marginH4 >0 | marginD4>0 | marginA4 >0) & (difference_to_scrape <= 5)')
odds_merged= oddschecker.merge(results,on= ['date','home_team','away_team'])
odds_merged.head()

,home_team,away_team,date,scrape_time_x,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,ToteA,scrape_time_y,PinnacleHome,PinnacleDraw,PinnacleAway,league_y,difference_to_scrape,ProbsPinnacleHome,ProbsPinnacleDraw,ProbsPinnacleAway
0,Genoa,Sassuolo,2021-05-09 12:30:00,2021-05-08 21:24:31.168019,3.4,3.40,2.1,3.35,3.45,2.15,...,NaN,2021-05-08 20:56:00,3.560,3.56,2.17,Italy - Serie A,15.566667,0.274684,0.274684,0.450633
1,CWKS Resovia Rzeszow,Widzew Lodz,2021-05-09 12:40:00,2021-05-08 21:26:44.707395,0.0,0.00,0.0,0.00,0.00,0.00,...,NaN,2021-05-08 21:14:00,3.040,3.26,2.48,Poland - 1st Liga,15.433333,0.316624,0.295257,0.388120
2,Meppen,Lubeck,2021-05-09 13:00:00,2021-05-08 21:23:40.178980,0.0,0.00,0.0,1.85,3.35,3.70,...,NaN,2021-05-08 21:08:00,1.909,3.53,4.26,Germany - Bundesliga,15.866667,0.502787,0.271904,0.225310
3,St Etienne,Marseille,2021-05-09 13:00:00,2021-05-08 21:23:48.340885,3.2,3.55,2.1,3.25,3.45,2.15,...,NaN,2021-05-08 21:07:00,3.400,3.71,2.18,France - Ligue,15.883333,0.287681,0.263643,0.448676
4,UT Arad,CSM Politehnica Iasi,2021-05-09 13:00:00,2021-05-08 21:26:48.997715,0.0,0.00,0.0,0.00,0.00,0.00,...,NaN,2021-05-08 21:15:00,1.476,4.20,7.73,Romania - Liga,15.750000,0.648352,0.227849,0.123799


In [151]:
odds_merged= odds_merged.drop_duplicates(subset= ['home_team','away_team']).reset_index(drop= True)
odds_merged.head()

,home_team,away_team,date,scrape_time_x,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,ToteA,scrape_time_y,PinnacleHome,PinnacleDraw,PinnacleAway,league_y,difference_to_scrape,ProbsPinnacleHome,ProbsPinnacleDraw,ProbsPinnacleAway
0,Genoa,Sassuolo,2021-05-09 12:30:00,2021-05-08 21:24:31.168019,3.4,3.40,2.1,3.35,3.45,2.15,...,NaN,2021-05-08 20:56:00,3.560,3.56,2.17,Italy - Serie A,15.566667,0.274684,0.274684,0.450633
1,CWKS Resovia Rzeszow,Widzew Lodz,2021-05-09 12:40:00,2021-05-08 21:26:44.707395,0.0,0.00,0.0,0.00,0.00,0.00,...,NaN,2021-05-08 21:14:00,3.040,3.26,2.48,Poland - 1st Liga,15.433333,0.316624,0.295257,0.388120
2,Meppen,Lubeck,2021-05-09 13:00:00,2021-05-08 21:23:40.178980,0.0,0.00,0.0,1.85,3.35,3.70,...,NaN,2021-05-08 21:08:00,1.909,3.53,4.26,Germany - Bundesliga,15.866667,0.502787,0.271904,0.225310
3,St Etienne,Marseille,2021-05-09 13:00:00,2021-05-08 21:23:48.340885,3.2,3.55,2.1,3.25,3.45,2.15,...,NaN,2021-05-08 21:07:00,3.400,3.71,2.18,France - Ligue,15.883333,0.287681,0.263643,0.448676
4,UT Arad,CSM Politehnica Iasi,2021-05-09 13:00:00,2021-05-08 21:26:48.997715,0.0,0.00,0.0,0.00,0.00,0.00,...,NaN,2021-05-08 21:15:00,1.476,4.20,7.73,Romania - Liga,15.750000,0.648352,0.227849,0.123799


In [152]:
bookies= ['10BetH', '10BetD',
       '10BetA', '888sportH', '888sportD', '888sportA', 'Bet VictorH',
       'Bet VictorD', 'Bet VictorA', 'Bet365H', 'Bet365D', 'Bet365A',
       'Betfair SportsbookH', 'Betfair SportsbookD', 'Betfair SportsbookA',
       'BetfairH', 'BetfairD', 'BetfairA', 'BetfredH', 'BetfredD', 'BetfredA',
       'BetwayH', 'BetwayD', 'BetwayA', 'Boyle SportsH', 'Boyle SportsD',
       'Boyle SportsA', 'GentingBetH', 'GentingBetD', 'GentingBetA',
       'Mansion BetH', 'Mansion BetD', 'Mansion BetA', 'MatchbookH',
       'MatchbookD', 'MatchbookA', 'NovibetH', 'NovibetD', 'NovibetA',
       'Paddy PowerH', 'Paddy PowerD', 'Paddy PowerA', 'RedzoneH', 'RedzoneD',
       'RedzoneA', 'SkybetH', 'SkybetD', 'SkybetA', 'Smarkets SportsbookH',
       'Smarkets SportsbookD', 'Smarkets SportsbookA', 'SmarketsH',
       'SmarketsD', 'SmarketsA', 'Sport NationH', 'Sport NationD',
       'Sport NationA', 'Sporting IndexH', 'Sporting IndexD',
       'Sporting IndexA', 'SpreadexH', 'SpreadexD', 'SpreadexA', 'UnibetH',
       'UnibetD', 'UnibetA', 'VBetH', 'VBetD', 'VBetA', 'William HillH',
       'William HillD', 'William HillA']

exchanges2= ['Betfair SportsbookH','Betfair SportsbookD','Betfair SportsbookA','BetfairH','BetfairD','BetfairA',\
            'MatchbookH','MatchbookD','MatchbookA','SmarketsH','SmarketsD','SmarketsA','Smarkets SportsbookH','Smarkets SportsbookD','Smarkets SportsbookA']


exchanges= ['BetfairH','BetfairD','BetfairA',\
            'MatchbookH','MatchbookD','MatchbookA','SmarketsH','SmarketsD','SmarketsA']
homes= [x for x in bookies if x.endswith('H')]
draws= [x for x in bookies if x.endswith('D')]
away= [x for x in bookies if x.endswith('A')]

homes2= [x for x in bookies if x.endswith('H') and x not in exchanges]
draws2= [x for x in bookies if x.endswith('D') and x not in exchanges]
away2= [x for x in bookies if x.endswith('A') and x not in exchanges ]

homes3= [x for x in bookies if x.endswith('H') and x in exchanges]
draws3= [x for x in bookies if x.endswith('D') and x in exchanges]
away3= [x for x in bookies if x.endswith('A') and x in exchanges ]

homes4= [x for x in bookies if x.endswith('H') and x not in exchanges2]
draws4= [x for x in bookies if x.endswith('D') and x not in exchanges2]
away4= [x for x in bookies if x.endswith('A') and x not in exchanges2]

In [153]:
odds_merged['time_lag']= (odds_merged.scrape_time_x - odds_merged.scrape_time_y).dt.total_seconds()
odds_merged['max_oddsH']= odds_merged[homes].max(axis= 1)
odds_merged['max_oddsD']= odds_merged[draws].max(axis= 1)
odds_merged['max_oddsA']= odds_merged[away].max(axis= 1)

odds_merged['max_oddsH2']= odds_merged[homes2].max(axis= 1)
odds_merged['max_oddsD2']= odds_merged[draws2].max(axis= 1)
odds_merged['max_oddsA2']= odds_merged[away2].max(axis= 1)

odds_merged['max_oddsH3']= odds_merged[homes3].max(axis= 1)
odds_merged['max_oddsD3']= odds_merged[draws3].max(axis= 1)
odds_merged['max_oddsA3']= odds_merged[away3].max(axis= 1)

odds_merged['max_oddsH4']= odds_merged[homes4].max(axis= 1)
odds_merged['max_oddsD4']= odds_merged[draws4].max(axis= 1)
odds_merged['max_oddsA4']= odds_merged[away4].max(axis= 1)

odds_merged['max_BookieH']= odds_merged[homes].idxmax(axis=1)
odds_merged['max_BookieD']= odds_merged[draws].idxmax(axis= 1)
odds_merged['max_BookieA']= odds_merged[away].idxmax(axis= 1)

odds_merged['max_BookieH2']= odds_merged[homes2].idxmax(axis=1)
odds_merged['max_BookieD2']= odds_merged[draws2].idxmax(axis= 1)
odds_merged['max_BookieA2']= odds_merged[away2].idxmax(axis= 1)

odds_merged['max_BookieH3']= odds_merged[homes3].idxmax(axis=1)
odds_merged['max_BookieD3']= odds_merged[draws3].idxmax(axis= 1)
odds_merged['max_BookieA3']= odds_merged[away3].idxmax(axis= 1)

odds_merged['max_BookieH4']= odds_merged[homes4].idxmax(axis=1)
odds_merged['max_BookieD4']= odds_merged[draws4].idxmax(axis= 1)
odds_merged['max_BookieA4']= odds_merged[away4].idxmax(axis= 1)

odds_merged['marginH']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged['marginH2']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH2']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD2']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD2']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA2']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA2']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged['marginH3']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH3']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD3']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD3']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA3']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA3']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged['marginH4']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH4']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD4']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD4']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA4']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA4']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged.head()

,home_team,away_team,date,scrape_time_x,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,marginA,marginH2,marginD2,marginA2,marginH3,marginD3,marginA3,marginH4,marginD4,marginA4
0,Genoa,Sassuolo,2021-05-09 12:30:00,2021-05-08 21:24:31.168019,3.4,3.40,2.1,3.35,3.45,2.15,...,-0.008608,-0.022127,-0.011139,-0.008608,-0.024873,-0.011139,-0.008608,-0.022127,-0.011139,-0.031139
1,CWKS Resovia Rzeszow,Widzew Lodz,2021-05-09 12:40:00,2021-05-08 21:26:44.707395,0.0,0.00,0.0,0.00,0.00,0.00,...,-0.126731,-0.091290,-0.084705,-0.126731,-1.000000,-1.000000,-1.000000,-0.091290,-0.084705,-0.126731
2,Meppen,Lubeck,2021-05-09 13:00:00,2021-05-08 21:23:40.178980,0.0,0.00,0.0,1.85,3.35,3.70,...,-0.098761,-0.019566,0.033234,-0.098761,-1.000000,-1.000000,-1.000000,-0.019566,0.033234,-0.098761
3,St Etienne,Marseille,2021-05-09 13:00:00,2021-05-08 21:23:48.340885,3.2,3.55,2.1,3.25,3.45,2.15,...,-0.012912,-0.007501,-0.027158,-0.012912,-0.007501,-0.032431,-0.012912,-0.021885,-0.027158,-0.012912
4,UT Arad,CSM Politehnica Iasi,2021-05-09 13:00:00,2021-05-08 21:26:48.997715,0.0,0.00,0.0,0.00,0.00,0.00,...,-0.071507,-0.008022,-0.065818,-0.071507,-0.046923,-0.154679,-0.099981,-0.008022,-0.065818,-0.071507


In [192]:
odds_merged[['home_team', 'away_team', 'date','PinnacleHome', 'PinnacleDraw', 'PinnacleAway', 
        'ProbsPinnacleHome', 'ProbsPinnacleDraw',
       'ProbsPinnacleAway', 'time_lag', 'max_oddsH', 'max_oddsD', 'max_oddsA',
       'max_BookieH', 'max_BookieD', 'max_BookieA', 'marginH', 'marginD',
       'marginA', 'max_BookieH2', 'max_BookieD2', 'max_BookieA2', 'marginH2', 'marginD2',
       'marginA2', 'marginH3', 'marginD3',
       'marginA3']].head()

,home_team,away_team,date,PinnacleHome,PinnacleDraw,PinnacleAway,ProbsPinnacleHome,ProbsPinnacleDraw,ProbsPinnacleAway,time_lag,...,marginA,max_BookieH2,max_BookieD2,max_BookieA2,marginH2,marginD2,marginA2,marginH3,marginD3,marginA3
0,Genoa,Sassuolo,2021-05-09 12:30:00,3.560547,3.560547,2.169922,0.274658,0.274658,0.450684,1711.0,...,-0.008606,VBetH,SkybetD,Smarkets SportsbookA,-0.022125,-0.011139,-0.008606,-0.024872,-0.011139,-0.008606
1,CWKS Resovia Rzeszow,Widzew Lodz,2021-05-09 12:40:00,3.039062,3.259766,2.480469,0.316650,0.295166,0.388184,764.5,...,-0.126709,Bet365H,Bet365D,Bet365A,-0.091309,-0.084717,-0.126709,-1.000000,-1.000000,-1.000000
2,Meppen,Lubeck,2021-05-09 13:00:00,1.909180,3.529297,4.261719,0.502930,0.271973,0.225342,940.0,...,-0.098755,William HillH,Bet365D,UnibetA,-0.019562,0.033234,-0.098755,-1.000000,-1.000000,-1.000000
3,St Etienne,Marseille,2021-05-09 13:00:00,3.400391,3.710938,2.179688,0.287598,0.263672,0.448730,1008.5,...,-0.012909,Smarkets SportsbookH,VBetD,NovibetA,-0.007500,-0.027161,-0.012909,-0.007500,-0.032440,-0.012909
4,UT Arad,CSM Politehnica Iasi,2021-05-09 13:00:00,1.475586,4.199219,7.730469,0.648438,0.227905,0.123779,709.0,...,-0.071533,Bet365H,Bet VictorD,Bet VictorA,-0.008018,-0.065796,-0.071533,-0.046936,-0.154663,-0.099976


In [155]:
attack= ['pinnacle','consens','cons_exch','cons_withou_exch','betfair','matchbook','']
defense= ['all', 'without_real_exchanges','without_exchanges', 'only exchanges']

combs= [(x,y) for x in attack for y in defense]
combs

[('pinnacle', 'all'),
 ('pinnacle', 'without_real_exchanges'),
 ('pinnacle', 'without_exchanges'),
 ('pinnacle', 'only exchanges'),
 ('consens', 'all'),
 ('consens', 'without_real_exchanges'),
 ('consens', 'without_exchanges'),
 ('consens', 'only exchanges'),
 ('cons_exch', 'all'),
 ('cons_exch', 'without_real_exchanges'),
 ('cons_exch', 'without_exchanges'),
 ('cons_exch', 'only exchanges'),
 ('cons_withou_exch', 'all'),
 ('cons_withou_exch', 'without_real_exchanges'),
 ('cons_withou_exch', 'without_exchanges'),
 ('cons_withou_exch', 'only exchanges'),
 ('betfair', 'all'),
 ('betfair', 'without_real_exchanges'),
 ('betfair', 'without_exchanges'),
 ('betfair', 'only exchanges'),
 ('matchbook', 'all'),
 ('matchbook', 'without_real_exchanges'),
 ('matchbook', 'without_exchanges'),
 ('matchbook', 'only exchanges'),
 ('', 'all'),
 ('', 'without_real_exchanges'),
 ('', 'without_exchanges'),
 ('', 'only exchanges')]

In [157]:
#test= odds_merged.marginA.astype('float16')
floats= odds_merged.select_dtypes('float').columns

odds_merged[floats]= odds_merged[floats].astype('float16')

odds_merged.info()
odds_merged.to_csv('odds_merged.csv',mode= 'a',index= False,header= False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Columns: 126 entries, home_team to marginA4
dtypes: datetime64[ns](3), float16(107), object(16)
memory usage: 50.5+ KB


In [175]:
test= pd.read_csv('odds_merged.csv')

#odds_merged
test['date']= pd.to_datetime(test['date'])

In [183]:
#test.sort_values('date')
odds_merged[:1].to_csv('odds_merged.csv',mode= 'a',index= False,header= False)

In [191]:
test= pd.read_csv('odds_merged.csv')
test['date']= pd.to_datetime(test['date'])


*Make it more efficient*
1. Fix the dates that have month and days interchanged!! --> FIND THEM ALL 
2. Alway read csv into python when appending !!! Don't append to the csv directly, since this is prone to errors 
3. Store the data on a sql database 
